# Korg for GALAH

In [84]:
try:
    %matplotlib inline
    %config InlineBackend.figure_format='retina'
except:
    pass

# Korg and Julia
from juliacall import Main as jl
jl.seval("using Korg")
Korg = jl.Korg

# general packages
import time
import numpy as np
from astropy.io import fits
from astropy.table import Table
from scipy.io import readsav

# matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

In [167]:
# Prepare HERMES information
galah_lines = Korg.get_GALAH_DR3_linelist()

In [166]:
def get_spectrum(sobject_id):
    
    spectrum = dict()
    spectrum['sobject_id'] = sobject_id
    
    for ccd in [1,2,3,4]:
        
        fits_file = fits.open('/Users/buder/GALAH_DR4/observations/'+str(sobject_id)[:6]+'/spectra/com/'+str(sobject_id)+str(ccd)+'.fits')
        spectrum['wave_air_ccd'+str(ccd)] = fits_file[0].header['CRVAL1'] + fits_file[0].header['CDELT1'] * np.arange(fits_file[0].header['NAXIS1'])
        spectrum['counts_ccd'+str(ccd)] = fits_file[0].data
        spectrum['counts_unc_ccd'+str(ccd)] = fits_file[0].data * fits_file[2].data
        spectrum['flux_norm_ccd'+str(ccd)] = fits_file[1].data
        spectrum['flux_unc_norm_ccd'+str(ccd)] = fits_file[1].data * fits_file[2].data
        fits_file.close()
        
    return(spectrum)

spectrum = get_spectrum(sobject_id = 210115002201239)

In [18]:
wavelengths_synthetic = dict()
LSF_matrix = dict()

synthetic_ranges = jl.seval('[4713:0.01:4903, 5648:0.01:5873, 6478:0.01:6737, 7585:0.01:7887]')
observed_ranges = np.concatenate(([[Korg.air_to_vacuum(λ) for λ in spectrum['wave_air_ccd'+str(ccd)]] for ccd in [1,2,3,4]]))

LSF_matrix_all = Korg.compute_LSF_matrix(
    synthetic_ranges,
    observed_ranges,
    28_000)

Progress:  21%|████████▌              

In [155]:
def compute_spectrum(
    teff,
    logg,
    fe_h,
    c_fe,
    n_fe,
    observed_ranges
):
    # Create the A(X) dictionary for synthesising lines with Korg
    alpha_fe = -0.4 * fe_h
    if alpha_fe > 0.4: alpha_fe = 0.4
    if alpha_fe < 0.0: alpha_fe = 0.0
    alpha_h = alpha_fe + fe_h
    c_h = c_fe + fe_h
    n_h = n_fe + fe_h
    a_x_dictionary = Korg.format_A_X(fe_h, alpha_h,{"C": c_h, "N": n_h})
    
    params = {'teff':teff, 'logg':logg, 'fe_h':fe_h, 'vsini':5.0, 'epsilon':0.6}
    
    # Interpolate the atmosphere
    atmosphere = Korg.interpolate_marcs(teff, logg, a_x_dictionary)
    
    # Synthesise the Korg spectrum
    synthesis = Korg.synthesize(atmosphere, lines, a_x_dictionary, synthetic_ranges)
    
    F = np.array(synthesis.flux)/np.array(synthesis.cntm)
    F = Korg.apply_rotation(F, synthetic_ranges, params["vsini"], params["epsilon"])
    F = LSF_matrix_all * F
    
    return(synthesis, np.array(F))

In [ ]:
t = time.process_time()
synthesis1, s1 = compute_spectrum(
    5772, # teff
    4.44, # logg
    0.00, # fe_h
    0.00, # c_fe
    0.00, # n_fe
    observed_ranges
)
print(time.process_time() - t)

In [ ]:
fit_result = Korg.Fit.fit_spectrum(
    # wavelength in vacuum
    np.concatenate(([[Korg.air_to_vacuum(λ) for λ in spectrum['wave_air_ccd'+str(ccd)]] for ccd in [1,2,3,4]])),
    # normalised flux
    np.concatenate(([[f in spectrum['flux_norm_ccd'+str(ccd)]] for ccd in [1,2,3,4]])),
    # normalised flux uncertainty
    np.concatenate(([[f in spectrum['flux_unc_norm_ccd'+str(ccd)]] for ccd in [1,2,3,4]])),
    galah_lines,
    initial_guess = (;Teff=5400, logg=3.8, m_H=-1.1, vmic=1.0);
    windows=winds,
    R=28_000
)